## Extracting checkpoint,bronze, silver containers URLs

In [0]:
checkpoint_location = spark.sql(""" DESCRIBE EXTERNAL LOCATION `checkpoints` """).select("url").collect()[0][0]
bronze_location = spark.sql(""" DESCRIBE EXTERNAL LOCATION `bronze` """).select("url").collect()[0][0]
silver_location = spark.sql(""" DESCRIBE EXTERNAL LOCATION `silver` """).select("url").collect()[0][0]

In [0]:
dbutils.widgets.text(name = "environment", defaultValue = "", label = "please type environment")
current_environment = dbutils.widgets.get("environment")

## Read Data from Bronze Table

In [0]:
def read_bronzeTraffic(env):
    print("Reading Bronze Traffic Data:  '", end='')
    df_bronzeTraffic =( spark.readStream
                          .table(f"`{env}_catalog`.`bronze`.`raw_traffic`")
                          )
    print(f'Reading {env}_catalog`.`bronze`.`raw_traffic` successfully read!!')
    return df_bronzeTraffic


## Handling duplicated rows

In [0]:
def remove_dups(df):
  print("Removing Duplicates values:  '", end='')
  df_remove_dups = df.dropDuplicates()
  print('Success!!  ')
  return df_remove_dups

## Handling null values by replacing them


In [0]:
def handle_NULLS(df, columns):
  print("Handling NULL values:  '", end='')
  print('Replacing NuLL values on String Columns with "UnKnown ', end='')
  df_string = df.fillna('UnKnown', subset=columns)
  print('Success!!  ')

  print('Replacing NuLL values on Numeric Columns with "0" ', end='')
  df_clean = df.fillna(0, subset=columns)
  print('Success!!  ')

  return df_clean

### Getting count of Electic vehicles by creating new column



In [0]:

def ev_Vehicles_count(df):
    print('creating Electric vehicles count column : ', end='')    
    from pyspark.sql.functions import col
    df_ev_vehicles = df.withColumn("Electic_vehicles_count",
                                col('Ev_Car') + col('EV_Bike')
                                )
    print('success!! ')            
    return df_ev_vehicles

## Creating function to get count of all motor vehicles

In [0]:
def motor_Vehicles_count(df):
    print('creating All Motor vehicles count column : ', end='')    
    from pyspark.sql.functions import col
    df_motor_vehicles = df.withColumn("Motor_vehicles_count",
                                col('Electic_vehicles_count') + col('Two_wheeled_motor_vehicles') + col('Cars_and_taxis') + col('Buses_and_coaches') + col('LGV_Type') + col('HGV_Type'))
    print('success!! ')            
    return df_motor_vehicles

## Creating Transformed Time Column

In [0]:
def create_Transformedtime(df):
    from pyspark.sql.functions import current_timestamp
    print('creating transformed time column: ', end='')
    df_add_time_stamp = df.withColumn("Transformed_Time",
                                   current_timestamp()
                                   )
    print('success !!')                             
    return df_add_time_stamp



## writing the Transformed data to Silver_Traffic_Table

In [0]:
def write_Traffic_SilverTable(df,environment):
    print('writing to silver table: ', end='')
    
    write_StreamSilver = (df.writeStream \
        .format("delta") \
        .option("checkpointLocation", checkpoint_location + "/SilverTrafficLoad/checkpt/") \
        .outputMode('append') \
        .queryName('silverTrafficWriteStream') \
        .trigger(availableNow=True)       
        .toTable(f"`{environment}_catalog`.`silver`.`silver_traffic`"))
    
    write_StreamSilver.awaitTermination()
    print(f'writing {environment}_catalog`.`silver`.`silver_traffic`") ')     

## calling the function

In [0]:
## Reading the bronze traffic data 
df_traffic_table_data = read_bronzeTraffic(current_environment)

## To remove duplicate rows
df_dups = remove_dups(df_traffic_table_data)

## To replace anu NULL values
All_columns_bronzetraffic = df_dups.schema.names
df_nulls = handle_NULLS(df_dups, All_columns_bronzetraffic)

## To get the total Ev_Count
df_ev = ev_Vehicles_count(df_nulls)

## To get the total Motor Vehicles
df_motor_count = motor_Vehicles_count(df_ev)

#calling Transformed time function
df_final = create_Transformedtime(df_motor_count)

## writing to silver traffic
write_Traffic_SilverTable(df_final,current_environment)

In [0]:
%sql
SELECT COUNT(0) FROM `dev_catalog`.`silver`.`silver_traffic`

In [0]:
%sql 
SELECT * FROM `dev_catalog`.`silver`.`silver_traffic`
WHERE Record_ID BETWEEN '37086' and '37095'
ORDER BY Record_ID